# Вариант 4. Васильев Максим, 6373.

## Задача 1.

### Текст задачи 

В цехах $N1$ и $N2$ предприятия производится продукт Y, который в дальнейшем используется в качестве исходного материала для производства изделий в цехе $N3$. Суммарная производительность цехов N1 и N2 зависит от вложения дополнительных средств X. При работе цехов N1 и N2 в течение одного месяца эта зависимость может быть приближённо представлена в виде функций:

- $N1: y = 5 + (x + 40)^{1/3}$
- $N2: y = 7 + (x + 30)^{1/3}$

Функции остатка средств в течении месяца:

- $N1: 0.8x$
- $N2: 0.62x$

Средства выделяемые на оба цеха в течении квартала (3 месяца), составляют 179 единиц; перераспределение производится помесячно. 

Требуется распределить средства на планируемый квартал с целью получения максимального количества продукта Y.

### Формализация задачи

Так-как количество управлений невелико, для решения задачи был выбран метод Динамического программирования.Для решения задачи выбранным методом, нужно выбрать способ описания процесса.

#### Способ описания процесса

1. Этапы   - месяца финансирования, $i = 0..2$
2. Выигрыш - суммарная производительность двух цехов, $y = 12 + (x_1 + 40)^{1/3} + (x_2 + 30)^{1/3}$, где $x_1$ - вложерние в первый цех, $x_2$ - вложение во второй цех
3. Управление - количество средств выделенное на первый цех, т.к. на второй цех будет выделена автоматически оставшаяся часть.
4. Состояние - остаток средств в течении месяца, в первом месяце это $179$

#### Описание в терминах уравнения Беллмана

$S_i$ - состояние на $i$-м этапе  
$u_i$ - управление на $i$-м этапе  
$W_i$ - условный оптимальный выигрыш на всех шагах от i-го и до последнего  
$w_i(S_i, u_i)$ - выигрыш на $i$-м этапе  
$\varphi_i(S_i, u_i)$ - изменение состояния системы на i-м шаге  

Запишем выигрыш и изменение состояния на i-м шаге:  
$w_i(S_i, u_i) = 12 + (u_i + 40)^{1/3} + (S_i - u_i + 30)^{1/3}$  
$\varphi_i(S_i, u_i) = 0.8 u_i + 0.66(S_i - u_i)$  

Тогда основное функциональное уравнение будет иметь следующий вид:

$W_i(S_i) = max(12 + (u_i + 40)^{1/3} + (S_i - u_i + 30)^{1/3} + W_{i+1}(0.66S_i + 0.14u_i))$

In [34]:
# Utility decorators 

def benchmark(func):
    # TODO: Only works right for non-recursive functions
    def decorated_function(*args, **kwargs):
        start = default_timer()
        ret = func(*args, *kwargs)
        end = default_timer()
        print(f"Execution time = {end - start}")
        return ret
    return decorated_function

def cached_function(func):
    cache = {}
    def decorated_function(*args, **kwargs):
        key = tuple(args)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return decorated_function



In [37]:
# Dynamic programming model
from typing import Callable, Union, Tuple, List


class DynamicSolver:
    def __init__(self,
                 state_change: Callable[[float, float], float],
                 local_profit_change: Callable[[float, float], float],
                 max_stages: int):
        self.state_change = cached_function(state_change)
        self.local_profit_change = cached_function(local_profit_change)
        self.max_stages = max_stages
    
    @cached_function
    def global_profit(self, stage: int, state: Union[int, float]) -> Tuple[float, int]:
        if stage >= self.max_stages:
            return (0, None)
        
        # TODO: Rewrite code below using itertools and change function constraints
        state = int(state)
        profit = [
            (
                self.local_profit_change(state, management) + self.global_profit(stage + 1, self.state_change(state, management))[0],
                management
            ) for management in range(state + 1)
        ]
        return max(profit, key=lambda x: x[0])
    

local_win = lambda s, u: 12 + (u + 40) ** (1/3) + ((s - u) + 30) ** (1/3)
local_state_change = lambda s, u: 0.8 * u + 0.66 * (s - u)

cost_optimizer = DynamicSolver(local_state_change, local_win, 3)
print(cost_optimizer.global_profit(0, 179))


(64.02499196970392, 107)
